In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torch

from utils.data import get_hsm_dataset, split_data, log_returns, get_solar_energy_dataset, get_fuel_prices_dataset, get_passengers_dataset, DimUniversalStandardScaler
from utils.metrics import MAPE, WAPE, MAE

from fourier_flows.SequentialFlows import FourierFlow, RealNVP, TimeFlow

In [2]:
hsm_dataset_path = Path("data/huge_stock_market_dataset/")
solar_energy_dataset_path = Path("data/solar_energy/")
fuel_prices_dataset_path = Path("data/fuel_prices/")
passengers_dataset_path = Path("data/air_passengers/")
models_dir = Path("models/")

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

val_size = 0.0
test_size = 0.0

n_samples = 80  # number of samples generated by QuantGAN

cuda:0


# Fourier Flow

In [4]:
ts_iterator = get_hsm_dataset(hsm_dataset_path, selected_files=hsm_dataset_path / "selected100.csv")
synthetic_path = hsm_dataset_path / "synthetic/FourierFlow/"

for ts_index, time_series in enumerate(ts_iterator):
    print(f"Time Series #{ts_index}")
    
    # train_ts = log_returns(time_series)
    train_ts = time_series
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]
    
    scaler = DimUniversalStandardScaler()
    train_ts = scaler.fit_transform(train_ts)

    torch.random.manual_seed(0)
    FF_model = FourierFlow(hidden=200, fft_size=len(train_ts), n_flows=10, normalize=False)

    FF_losses = FF_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = scaler.inverse_transform(FF_model.sample(n_samples)) 
    np.save(synthetic_path / f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, FF_model, FF_losses

Time Series #0
step: 0 	/ 50 	-	loss: 1433.762
step: 49 	/ 50 	|	loss: -8684.314
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 920.409
step: 49 	/ 50 	|	loss: -5870.284
Finished training!
Time Series #2
step: 0 	/ 50 	-	loss: 918.597
step: 49 	/ 50 	|	loss: -5676.129
Finished training!
Time Series #3
step: 0 	/ 50 	-	loss: 1438.518
step: 49 	/ 50 	|	loss: -8718.995
Finished training!
Time Series #4
step: 0 	/ 50 	-	loss: 3636.577
step: 49 	/ 50 	|	loss: -23358.168
Finished training!
Time Series #5
step: 0 	/ 50 	-	loss: 446.304
step: 49 	/ 50 	|	loss: -2438.261
Finished training!
Time Series #6
step: 0 	/ 50 	-	loss: 1256.994
step: 49 	/ 50 	|	loss: -7797.780
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 661.016
step: 49 	/ 50 	|	loss: -3615.627
Finished training!
Time Series #8
step: 0 	/ 50 	-	loss: 3204.709
step: 49 	/ 50 	|	loss: -21382.936
Finished training!
Time Series #9
step: 0 	/ 50 	-	loss: 395.522
step: 49 	/ 50 	|	loss: -1955.700
Finished training!
Tim

Time: 13 min 22 sec - 100 ts

In [4]:
ts_iterator = get_solar_energy_dataset(solar_energy_dataset_path)
synthetic_path = solar_energy_dataset_path / "synthetic/FourierFlow/"
start_point = 0
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start=start_point):
    print(f"Time Series #{ts_index}")
    
    train_ts = time_series
    # train_ts = log_returns(train_ts + 1e-9)
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]
    
    scaler = DimUniversalStandardScaler()
    train_ts = scaler.fit_transform(train_ts)

    torch.random.manual_seed(0)
    FF_model = FourierFlow(hidden=200, fft_size=len(train_ts), n_flows=10, normalize=False)

    FF_losses = FF_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = scaler.inverse_transform(FF_model.sample(n_samples))
    np.save(synthetic_path / f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, FF_model, FF_losses

Time Series #0
step: 0 	/ 50 	-	loss: 3772.498
step: 49 	/ 50 	|	loss: -23432.711
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 3772.984
step: 49 	/ 50 	|	loss: -22944.359
Finished training!
Time Series #2
step: 0 	/ 50 	-	loss: 3772.801
step: 49 	/ 50 	|	loss: -25263.359
Finished training!
Time Series #3
step: 0 	/ 50 	-	loss: 3772.489
step: 49 	/ 50 	|	loss: -25117.824
Finished training!
Time Series #4
step: 0 	/ 50 	-	loss: 3772.789
step: 49 	/ 50 	|	loss: -24623.168
Finished training!
Time Series #5
step: 0 	/ 50 	-	loss: 3773.261
step: 49 	/ 50 	|	loss: -23333.354
Finished training!
Time Series #6
step: 0 	/ 50 	-	loss: 3772.315
step: 49 	/ 50 	|	loss: -25599.855
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 3772.997
step: 49 	/ 50 	|	loss: -25447.047
Finished training!
Time Series #8
step: 0 	/ 50 	-	loss: 3772.715
step: 49 	/ 50 	|	loss: -24500.773
Finished training!
Time Series #9
step: 0 	/ 50 	-	loss: 3772.728
step: 49 	/ 50 	|	loss: -25117.707
Finished 

Time: 80 min 20 ts 10k points; 15 min 30 ts 3k

In [8]:
ts_iterator = get_fuel_prices_dataset(fuel_prices_dataset_path)
synthetic_path = fuel_prices_dataset_path / "synthetic/FourierFlow/"

for ts_index, time_series in enumerate(ts_iterator):
    print(f"Time Series #{ts_index}")
    
    # train_ts = log_returns(time_series + 1e-9)
    train_ts = time_series
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]
    
    scaler = DimUniversalStandardScaler()
    train_ts = scaler.fit_transform(train_ts)

    torch.random.manual_seed(0)
    FF_model = FourierFlow(hidden=200, fft_size=len(train_ts), n_flows=10, normalize=False)

    FF_losses = FF_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = scaler.inverse_transform(FF_model.sample(n_samples))
    np.save(synthetic_path / f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, FF_model, FF_losses

Time Series #0
step: 0 	/ 50 	-	loss: 1005.097
step: 49 	/ 50 	|	loss: -6032.013
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 1004.961
step: 49 	/ 50 	|	loss: -6121.559
Finished training!
Time Series #2
step: 0 	/ 50 	-	loss: 1005.282
step: 49 	/ 50 	|	loss: -6020.570
Finished training!
Time Series #3
step: 0 	/ 50 	-	loss: 1005.098
step: 49 	/ 50 	|	loss: -6248.771
Finished training!
Time Series #4
step: 0 	/ 50 	-	loss: 1004.573
step: 49 	/ 50 	|	loss: -6004.806
Finished training!
Time Series #5
step: 0 	/ 50 	-	loss: 1004.946
step: 49 	/ 50 	|	loss: -5832.396
Finished training!
Time Series #6
step: 0 	/ 50 	-	loss: 1076.497
step: 49 	/ 50 	|	loss: -4847.686
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 1076.265
step: 49 	/ 50 	|	loss: -4837.196
Finished training!


In [11]:
ts_iterator = get_passengers_dataset(passengers_dataset_path, max_results=50)
synthetic_path = passengers_dataset_path / "synthetic/FourierFlow/"

for ts_index, time_series in enumerate(ts_iterator):
    print(f"Time Series #{ts_index}")
    
    # train_ts = log_returns(time_series + 1e-9)
    train_ts = time_series
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]
    
    scaler = DimUniversalStandardScaler()
    train_ts = scaler.fit_transform(train_ts)

    torch.random.manual_seed(0)
    FF_model = FourierFlow(hidden=200, fft_size=len(train_ts), n_flows=10, normalize=False)

    FF_losses = FF_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = scaler.inverse_transform(FF_model.sample(n_samples))
    np.save(synthetic_path / f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, FF_model, FF_losses

Time Series #0
step: 0 	/ 50 	-	loss: 305.682
step: 49 	/ 50 	|	loss: -1279.137
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 304.775
step: 49 	/ 50 	|	loss: -1263.591
Finished training!
Time Series #2
step: 0 	/ 50 	-	loss: 295.371
step: 49 	/ 50 	|	loss: -1274.889
Finished training!
Time Series #3
step: 0 	/ 50 	-	loss: 292.092
step: 49 	/ 50 	|	loss: -1164.276
Finished training!
Time Series #4
step: 0 	/ 50 	-	loss: 292.464
step: 49 	/ 50 	|	loss: -1077.995
Finished training!
Time Series #5
step: 0 	/ 50 	-	loss: 304.051
step: 49 	/ 50 	|	loss: -1352.430
Finished training!
Time Series #6
step: 0 	/ 50 	-	loss: 305.294
step: 49 	/ 50 	|	loss: -1403.936
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 304.940
step: 49 	/ 50 	|	loss: -1343.296
Finished training!
Time Series #8
step: 0 	/ 50 	-	loss: 270.701
step: 49 	/ 50 	|	loss: -1065.688
Finished training!
Time Series #9
step: 0 	/ 50 	-	loss: 277.865
step: 49 	/ 50 	|	loss: -987.377
Finished training!
Time Series

# RealNVP

In [12]:
ts_iterator = get_hsm_dataset(hsm_dataset_path, selected_files=hsm_dataset_path / "selected100.csv")
synthetic_path = hsm_dataset_path / "synthetic/RealNVP/"

for ts_index, time_series in enumerate(ts_iterator):
    print(f"Time Series #{ts_index}")
    
    # train_ts = log_returns(time_series)
    train_ts = time_series
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]
    
    scaler = DimUniversalStandardScaler()
    train_ts = scaler.fit_transform(train_ts)

    torch.random.manual_seed(0)
    RealNVP_model = RealNVP(hidden=200, T=len(train_ts), n_flows=10, normalize=False)

    RealNVP_losses = RealNVP_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = scaler.inverse_transform(RealNVP_model.sample(n_samples))
    np.save(synthetic_path / f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, RealNVP_model, RealNVP_losses

Time Series #0
step: 0 	/ 50 	-	loss: 2857.240
step: 49 	/ 50 	|	loss: -2579.293
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 2064.856
step: 49 	/ 50 	|	loss: -1671.642
Finished training!
Time Series #2
step: 0 	/ 50 	-	loss: 1982.346
step: 49 	/ 50 	|	loss: -1724.481
Finished training!
Time Series #3
step: 0 	/ 50 	-	loss: 2913.241
step: 49 	/ 50 	|	loss: -2359.355
Finished training!
Time Series #4
step: 0 	/ 50 	-	loss: 6622.062
step: 49 	/ 50 	|	loss: -5033.756
Finished training!
Time Series #5
step: 0 	/ 50 	-	loss: 789.290
step: 49 	/ 50 	|	loss: -837.398
Finished training!
Time Series #6
step: 0 	/ 50 	-	loss: 2402.614
step: 49 	/ 50 	|	loss: -1981.542
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 1329.542
step: 49 	/ 50 	|	loss: -1095.135
Finished training!
Time Series #8
step: 0 	/ 50 	-	loss: 6534.726
step: 49 	/ 50 	|	loss: -6245.627
Finished training!
Time Series #9
step: 0 	/ 50 	-	loss: 733.632
step: 49 	/ 50 	|	loss: -718.795
Finished training!
Time

Time: 12:02

In [5]:
ts_iterator = get_solar_energy_dataset(solar_energy_dataset_path)
synthetic_path = solar_energy_dataset_path / "synthetic/RealNVP/"
start_point = 0
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start_point):
    print(f"Time Series #{ts_index}")
    
    train_ts = time_series
    # train_ts = log_returns(train_ts + 1e-9)
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]
    
    scaler = DimUniversalStandardScaler()
    train_ts = scaler.fit_transform(train_ts)

    torch.random.manual_seed(0)
    RealNVP_model = RealNVP(hidden=200, T=len(train_ts), n_flows=10, normalize=False)

    RealNVP_losses = RealNVP_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = scaler.inverse_transform(RealNVP_model.sample(n_samples))
    np.save(synthetic_path / f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, RealNVP_model, RealNVP_losses

Time Series #0
step: 0 	/ 50 	-	loss: 7581.825
step: 49 	/ 50 	|	loss: -6289.493
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 7535.469
step: 49 	/ 50 	|	loss: -6583.566
Finished training!
Time Series #2
step: 0 	/ 50 	-	loss: 7539.304
step: 49 	/ 50 	|	loss: -6191.801
Finished training!
Time Series #3
step: 0 	/ 50 	-	loss: 7547.858
step: 49 	/ 50 	|	loss: -6666.374
Finished training!
Time Series #4
step: 0 	/ 50 	-	loss: 7582.874
step: 49 	/ 50 	|	loss: -5549.914
Finished training!
Time Series #5
step: 0 	/ 50 	-	loss: 7615.954
step: 49 	/ 50 	|	loss: -6287.205
Finished training!
Time Series #6
step: 0 	/ 50 	-	loss: 7548.625
step: 49 	/ 50 	|	loss: -5487.417
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 7609.505
step: 49 	/ 50 	|	loss: -6550.481
Finished training!
Time Series #8
step: 0 	/ 50 	-	loss: 7561.396
step: 49 	/ 50 	|	loss: -5967.575
Finished training!
Time Series #9
step: 0 	/ 50 	-	loss: 7614.682
step: 49 	/ 50 	|	loss: -6269.918
Finished training!


Time: ~35 min 10 ts 10k; ~14 min 30 ts 3k

In [5]:
ts_iterator = get_fuel_prices_dataset(fuel_prices_dataset_path)
synthetic_path = fuel_prices_dataset_path / "synthetic/RealNVP/"
start_point = 0
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start_point):
    print(f"Time Series #{ts_index}")
    
    train_ts = time_series
    # train_ts = log_returns(train_ts + 1e-9)
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]
    
    scaler = DimUniversalStandardScaler()
    train_ts = scaler.fit_transform(train_ts)

    torch.random.manual_seed(0)
    RealNVP_model = RealNVP(hidden=200, T=len(train_ts), n_flows=10, normalize=False)

    RealNVP_losses = RealNVP_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = scaler.inverse_transform(RealNVP_model.sample(n_samples))

    del train_ts, synth_data, RealNVP_model, RealNVP_losses

Time Series #0
step: 0 	/ 50 	-	loss: 2095.259
step: 49 	/ 50 	|	loss: -1998.698
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 2219.909
step: 49 	/ 50 	|	loss: -1927.526
Finished training!
Time Series #2
step: 0 	/ 50 	-	loss: 2215.981
step: 49 	/ 50 	|	loss: -1757.854
Finished training!
Time Series #3
step: 0 	/ 50 	-	loss: 2273.755
step: 49 	/ 50 	|	loss: -1915.891
Finished training!
Time Series #4
step: 0 	/ 50 	-	loss: 2239.323
step: 49 	/ 50 	|	loss: -1859.007
Finished training!
Time Series #5
step: 0 	/ 50 	-	loss: 2323.543
step: 49 	/ 50 	|	loss: -1864.432
Finished training!
Time Series #6
step: 0 	/ 50 	-	loss: 2266.748
step: 49 	/ 50 	|	loss: -1922.500
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 2254.252
step: 49 	/ 50 	|	loss: -1616.521
Finished training!


In [6]:
ts_iterator = get_passengers_dataset(passengers_dataset_path)
synthetic_path = passengers_dataset_path / "synthetic/RealNVP/"
start_point = 0
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start_point):
    print(f"Time Series #{ts_index}")
    
    train_ts = time_series
    # train_ts = log_returns(train_ts + 1e-9)
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]
    
    scaler = DimUniversalStandardScaler()
    train_ts = scaler.fit_transform(train_ts)

    torch.random.manual_seed(0)
    RealNVP_model = RealNVP(hidden=200, T=len(train_ts), n_flows=10, normalize=False)

    RealNVP_losses = RealNVP_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = scaler.inverse_transform(RealNVP_model.sample(n_samples))

    del train_ts, synth_data, RealNVP_model, RealNVP_losses

Time Series #0
step: 0 	/ 50 	-	loss: 521.471
step: 49 	/ 50 	|	loss: -580.076
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 569.181
step: 49 	/ 50 	|	loss: -445.185
Finished training!
Time Series #2
step: 0 	/ 50 	-	loss: 452.948
step: 49 	/ 50 	|	loss: -447.992
Finished training!
Time Series #3
step: 0 	/ 50 	-	loss: 510.186
step: 49 	/ 50 	|	loss: -496.036
Finished training!
Time Series #4
step: 0 	/ 50 	-	loss: 513.866
step: 49 	/ 50 	|	loss: -424.895
Finished training!
Time Series #5
step: 0 	/ 50 	-	loss: 588.410
step: 49 	/ 50 	|	loss: -524.433
Finished training!
Time Series #6
step: 0 	/ 50 	-	loss: 585.592
step: 49 	/ 50 	|	loss: -476.250
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 572.464
step: 49 	/ 50 	|	loss: -467.996
Finished training!
Time Series #8
step: 0 	/ 50 	-	loss: 806.641
step: 49 	/ 50 	|	loss: -479.385
Finished training!
Time Series #9
step: 0 	/ 50 	-	loss: 458.834
step: 49 	/ 50 	|	loss: -518.970
Finished training!
Time Series #10
step

# TimeFlow

In [5]:
# synthetic_path = synthetic_path = f"{dataset_path}synthetic/TimeFlow/"
# ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")
# seed_everything(0)

# for ts_index, time_series in enumerate(ts_iterator):
#     print(f"Time Series #{ts_index}")
    
#     (train_ts, *_), *_ = split_data(time_series, val_size=val_size, test_size=test_size)
#     train_ts = log_returns(train_ts)
#     train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]

#     TimeFlow_model = TimeFlow(hidden=200, T=len(train_ts), n_flows=10, normalize=False)

#     TimeFlow_losses = TimeFlow_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
#                             learning_rate=1e-3, display_step=50)

#     synth_data = TimeFlow_model.sample(n_samples // len(train_ts))
#     np.save(synthetic_path + f"selected{ts_index}.npy", synth_data)

#     del train_ts, synth_data, TimeFlow_model, TimeFlow_losses

12 ts time: ~30 min

# Similarity

In [1]:
from tqdm import tqdm
from pathlib import Path

import numpy as np
import pandas as pd

from utils.data import get_hsm_dataset, get_solar_energy_dataset, get_fuel_prices_dataset, get_passengers_dataset, split_data, log_returns
from utils.synth_eval import eval_sim

In [2]:
results_dir = Path("results")
hsm_dataset_path = Path("data/huge_stock_market_dataset/")
solar_energy_dataset_path = Path("data/solar_energy/")
fuel_prices_dataset_path = Path("data/fuel_prices/")
passengers_dataset_path = Path("data/air_passengers/")
models_dir = Path("models/")

In [3]:
eval_sim(("hsm", "se", "fp", "ap"), (hsm_dataset_path, solar_energy_dataset_path, fuel_prices_dataset_path, passengers_dataset_path),
     "RealNVP", save=True, results_dir=results_dir)

processing hsm dataset


100it [00:21,  4.67it/s]


processing se dataset


10it [00:01,  6.81it/s]


processing fp dataset


8it [00:01,  5.57it/s]


processing ap dataset


50it [00:36,  1.37it/s]


defaultdict(dict,
            {'hsm': {'kl_div': 0.6693378168044315,
              'kstest_pval': 0.8745732782412585},
             'se': {'kl_div': 21.02509311160518, 'kstest_pval': 0.0},
             'fp': {'kl_div': 777.2237463919374, 'kstest_pval': 0.0},
             'ap': {'kl_div': 98847.20422903946,
              'kstest_pval': 3.5532512293679205e-108}})

In [4]:
eval_sim(("hsm", "se", "fp", "ap"), (hsm_dataset_path, solar_energy_dataset_path, fuel_prices_dataset_path, passengers_dataset_path),
     "FourierFlow", save=True, results_dir=results_dir)

processing hsm dataset


100it [00:21,  4.69it/s]


processing se dataset


10it [00:11,  1.10s/it]


processing fp dataset


8it [00:01,  7.01it/s]


processing ap dataset


50it [00:28,  1.75it/s]


defaultdict(dict,
            {'hsm': {'kl_div': 1.2821953428109116,
              'kstest_pval': 0.2712952688679027},
             'se': {'kl_div': 0.7233572757119009, 'kstest_pval': 5e-324},
             'fp': {'kl_div': 0.3183792738126374,
              'kstest_pval': 0.536908790499574},
             'ap': {'kl_div': 618.9773203555657,
              'kstest_pval': 0.5596898766054589}})